In [0]:
# p mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [0]:
 # load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation=augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image, shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
       # print("A")
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
        #    print("B")
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
58,resized/59.jpg,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
2106,resized/2107.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0
5206,resized/5207.jpg,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
1163,resized/1164.jpg,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
13534,resized/13536.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1


In [0]:
from imgaug import augmenters as iaa

aug1 = iaa.CoarseDropout(p=0.10, size_percent=0.05)

def additional_augmenation(image):
    image = aug1.augment_image(image)

    return image
## create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=ImageDataGenerator( 
                                    # randomly shift images horizontally 
                                    width_shift_range=0.1, 
                                    # randomly shift images vertically 
                                    height_shift_range=0.1, 
                                    # set range for random shear 
                                    shear_range=0.15, 
                                    # set range for random zoom 
                                    zoom_range=0., 
                                    # set range for random channel shifts 
                                    channel_shift_range=0., 
                                    # set mode for filling points outside the input boundaries 
                                    fill_mode='nearest', 
                                    # value used for fill_mode = "constant" 
                                    cval=0., 
                                    # randomly flip images 
                                    horizontal_flip=True, 
                                    # randomly flip images 
                                    vertical_flip=False, 
                                    # set function that will be applied on each input 
                                    preprocessing_function=additional_augmenation))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, Activation, BatchNormalization
from keras.models import Model
from keras.layers import Input
from keras.regularizers import l2
def convx(input_img, ch, ker1, ker2, type):
  x = Conv2D(ch, (ker1,ker2), padding=type, use_bias=False, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(input_img)
  x = BatchNormalization()(x)
  x =  Activation('relu')(x)
  return x 
def inception3a(input_img, var):
    tower_1 = convx(input_img, 64, 1, 1, 'valid')
    tower_1 = convx(tower_1, 96, 3, 3, 'same')
    tower_1 = convx(tower_1, 96, 3, 3, 'same')
    
    tower_2 = convx(input_img, 48, 1, 1, 'valid')
    tower_2 = convx(tower_2, 64, 5, 5, 'same')

    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = convx(tower_3, 64, 1, 1, 'valid')
    
    tower_4 = convx(input_img, 64, 1, 1, 'valid')
    
    output = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis = 3)
    return output  

def inception3b(input_img, var):

    tower_1 = convx(input_img, 128, 1, 1, 'valid')
    tower_1 = convx(tower_1, 128, 1, 7, 'same')
    tower_1 = convx(tower_1, 128, 7, 1, 'same')
    tower_1 = convx(tower_1, 128, 1, 7, 'same')
    tower_1 = convx(tower_1, 128, 7, 1, 'same')

    tower_2 = convx(input_img, 128, 1, 1, 'valid')
    tower_2 = convx(tower_2, 128, 1, 7, 'same')
    tower_2 = convx(tower_2, 192, 7, 1, 'same')
    
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = convx(tower_3, 192, 1, 1, 'valid')

    tower_4 = convx(input_img, 192, 1, 1, 'valid')

    output = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis = 3)
    return output  

def inception3c(input_img, var):
    tower_1 = convx(input_img, 448, 1, 1, 'valid')
    tower_1 = convx(tower_1, 384, 3, 3, 'same')
    tower_5 = convx(tower_1, 384, 3, 1, 'same')
    tower_1 = convx(tower_1, 384, 1, 3, 'same')
    output1 = keras.layers.concatenate([tower_1, tower_5], axis = 3)
    
    tower_2 = convx(input_img, 384, 1, 1, 'valid')
    tower_6 = convx(tower_2, 384, 1, 3, 'same')
    tower_2 = convx(tower_2, 384, 3, 1, 'same')
    output2 = keras.layers.concatenate([tower_2, tower_6], axis = 3)

    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = convx(tower_3, 192, 1, 1, 'valid')
    
    tower_4 = convx(input_img, 320, 1, 1, 'same')

    output = keras.layers.concatenate([output1, output2, tower_3, tower_4], axis = 3)
    return output  

def inception3d(input_img, var):
    
    tower_1 = Conv2D(384, (3,3), strides=2, use_bias=False, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(input_img)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)

    tower_2 = convx(input_img, 64, 1, 1, 'valid')
    tower_2 = convx(tower_2, 96, 3, 3, 'same')
    tower_2 = Conv2D(96, (3,3), strides=2, use_bias=False, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(tower_2)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)

    tower_3 = MaxPooling2D((3,3), strides=2)(input_img)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output  

def inception3e(input_img, var):
    tower_1 = convx(input_img, 192, 1, 1, 'valid')
    tower_1 = Conv2D(320, (3,3), strides=2, use_bias=False, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(input_img)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Activation('relu')(tower_1)

    tower_2 = convx(input_img, 192, 1, 1, 'valid')
    tower_2 = convx(tower_2, 192, 7, 1, 'same')
    tower_2 = convx(tower_2, 192, 1, 7, 'same')
    tower_2 = Conv2D(192, (3,3), strides=2, use_bias=False, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(tower_2)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Activation('relu')(tower_2)

    tower_3 = MaxPooling2D((3,3), strides=2)(input_img)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output 

#create model
input_img = Input(shape=(299,299,3))
x = Conv2D(32, (3,3), strides=2, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(input_img)
x = BatchNormalization()(x) 

x = convx(x, 32, 3, 3, 'same')
x = convx(x, 64, 3, 3, 'valid')
x = MaxPooling2D(pool_size = (3,3), strides=(2,2))(x)

x = convx(x, 80, 3, 3, 'same')
x = convx(x, 192, 3, 3, 'same')
x = MaxPooling2D(pool_size = (3,3), strides=(2,2))(x)

x = inception3a(x, 0)
x = inception3a(x, 0)
x = inception3a(x, 0)

x = inception3d(x, 0)

x = inception3b(x, 0)
x = inception3b(x, 0)
x = inception3b(x, 0)
x = inception3b(x, 0)

x = inception3e(x, 0)

x = inception3c(x, 0)
x = inception3c(x, 0)

x = AveragePooling2D(pool_size=7)(x)  
c = Dropout(0.25)(x) 
x = Flatten()(x)
x = Dense(512, activation="relu")(x)

def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.15)(neck)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(num_units[name], activation="softmax", name=f"{name}_output")(in_layer)

# heads 
gender = build_head("gender", build_tower(x))
image_quality = build_head("image_quality", build_tower(x))
age = build_head("age", build_tower(x))
weight = build_head("weight", build_tower(x))
bag = build_head("bag", build_tower(x))
footwear = build_head("footwear", build_tower(x))
emotion = build_head("emotion", build_tower(x))
pose = build_head("pose", build_tower(x))

outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]

#Instantiate model.
model1 = Model(inputs=input_img, outputs=outputs)

model1.summary()














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 149, 149, 32) 9216      

In [0]:
# freeze backbone
model = model1
for layer in model.layers:
	layer.trainable = True

In [0]:
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "mse",
 	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy"
 }
loss_weights = {"gender_output": 1.5, "image_quality_output": 1.5, "age_output": 2, "weight_output": 1.2, "bag_output": 1.2, "footwear_output": 1.5, "pose_output": 1.0, "emotion_output": 1.0}

In [0]:
from keras.callbacks import Callback
from keras import backend as K
import numpy as np

# Keras-Contib Implementation
class CyclicLR(Callback):

    def __init__(
            self,
            base_lr=0.0001,
            max_lr=0.0025,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2.**(x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        print("Learning Rate: ", float(K.get_value(self.model.optimizer.lr)))
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [0]:
def scheduler(epoch, lr):
    print("Learning rate", lr)
    return lr
    
opt = SGD(lr=0.001, momentum=0.9, nesterov=True, decay=0.0001)
model.compile(
    optimizer=opt,
    loss= losses, 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

from keras.callbacks import *
filepath="/content/gdrive/My Drive/MyCNN/epochs:{epoch:1}.hdf5"

lr_scheduler = LearningRateScheduler(scheduler)
clr = CyclicLR(base_lr=0.0005, 
               max_lr=0.006, 
               step_size=2000., 
               mode='triangular')

lr_reducer = ReduceLROnPlateau(monitor='loss',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)


csv_logger = CSVLogger('/content/gdrive/My Drive/MyCNN/log.csv', append=True, separator=',')
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, csv_logger, clr, lr_reducer, lr_scheduler]
#model.load_weights("/content/gdrive/My Drive/MyCNN/epochs:028-loss:11.585671-val_loss:11.766465.hdf5")
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    callbacks=callbacks_list,
    initial_epoch=0,
    workers=6, 
    epochs=500,
    verbose=1
)

Epoch 1/500
Learning rate 0.0005000000237487257
359/360 [============================>.] - ETA: 0s - loss: 194.8069 - gender_output_loss: 0.6914 - image_quality_output_loss: 0.9943 - age_output_loss: 0.1494 - weight_output_loss: 1.0134 - bag_output_loss: 0.9325 - footwear_output_loss: 1.0464 - pose_output_loss: 0.9452 - emotion_output_loss: 0.9287 - gender_output_acc: 0.5454 - image_quality_output_acc: 0.5435 - age_output_acc: 0.3965 - weight_output_acc: 0.6240 - bag_output_acc: 0.5555 - footwear_output_acc: 0.4422 - pose_output_acc: 0.6116 - emotion_output_acc: 0.7101

360/360 [==============================] - 165s 458ms/step - loss: 194.7679 - gender_output_loss: 0.6915 - image_quality_output_loss: 0.9943 - age_output_loss: 0.1494 - weight_output_loss: 1.0135 - bag_output_loss: 0.9326 - footwear_output_loss: 1.0465 - pose_output_loss: 0.9451 - emotion_output_loss: 0.9288 - gender_output_acc: 0.5453 - image_quality_output_acc: 0.5436 - age_output_acc: 0.3963 - weight_output_acc: 0.62

Process ForkPoolWorker-1137:
Process ForkPoolWorker-1133:
Process ForkPoolWorker-1139:
Process ForkPoolWorker-1138:
Traceback (most recent call last):
Process ForkPoolWorker-1140:
Process ForkPoolWorker-1131:
Process ForkPoolWorker-1135:
Process ForkPoolWorker-1136:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line

KeyboardInterrupt: ignored

In [0]:
def scheduler(epoch, lr):
    print("Learning rate", lr)
    return lr
    
opt = SGD(lr=0.001, momentum=0.9, nesterov=True, decay=0.0001)
model.compile(
    optimizer=opt,
    loss= losses, 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

from keras.callbacks import *
filepath="/content/gdrive/My Drive/MyCNN/epochs:{epoch:1}.hdf5"

lr_scheduler = LearningRateScheduler(scheduler)
clr = CyclicLR(base_lr=0.0005, 
               max_lr=0.002, 
               step_size=2000., 
               mode='triangular')

lr_reducer = ReduceLROnPlateau(monitor='loss',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)


csv_logger = CSVLogger('/content/gdrive/My Drive/MyCNN/log.csv', append=True, separator=',')
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, csv_logger, clr, lr_reducer, lr_scheduler]
model.load_weights("/content/gdrive/My Drive/MyCNN/epochs:90.hdf5")
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    callbacks=callbacks_list,
    initial_epoch=91,
    workers=6, 
    epochs=500,
    verbose=1
)

Epoch 92/500
Learning rate 0.0005000000237487257
360/360 [==============================] - 164s 455ms/step - loss: 6.4421 - gender_output_loss: 0.1601 - image_quality_output_loss: 0.8279 - age_output_loss: 0.1391 - weight_output_loss: 0.7938 - bag_output_loss: 0.6790 - footwear_output_loss: 0.6750 - pose_output_loss: 0.3111 - emotion_output_loss: 0.8270 - gender_output_acc: 0.9401 - image_quality_output_acc: 0.6049 - age_output_acc: 0.4246 - weight_output_acc: 0.6831 - bag_output_acc: 0.7289 - footwear_output_acc: 0.7087 - pose_output_acc: 0.8885 - emotion_output_acc: 0.7158 - val_loss: 7.4075 - val_gender_output_loss: 0.2940 - val_image_quality_output_loss: 0.8804 - val_age_output_loss: 0.1458 - val_weight_output_loss: 0.9002 - val_bag_output_loss: 0.7897 - val_footwear_output_loss: 0.8194 - val_pose_output_loss: 0.4169 - val_emotion_output_loss: 0.9099 - val_gender_output_acc: 0.8977 - val_image_quality_output_acc: 0.5675 - val_age_output_acc: 0.3735 - val_weight_output_acc: 0.6497 

Process ForkPoolWorker-1424:
Process ForkPoolWorker-1428:
Process ForkPoolWorker-1421:
Process ForkPoolWorker-1425:
Process ForkPoolWorker-1427:
Process ForkPoolWorker-1423:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1420:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-1418:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkPoolWorker-1426:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "

KeyboardInterrupt: ignored

In [0]:
a = []
while(1):
    a.append('1')